In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [ ]:
import optuna
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
import numpy as np
import mlflow
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor



# Parâmetros fixos dos modelos
params_xgboost = {
    'n_estimators': 1923,
    'learning_rate': 0.0176491247294085,
    'max_depth': 7,
    'min_child_weight': 1,
    'subsample': 0.7565637963621075,
    'reg_alpha': 2.4180671613496878e-06,
    'reg_lambda': 4.3213853733170415,
    'gamma': 0.0017610181848471591,
    'random_state': 42,
    'n_jobs': 1,
    'tree_method': 'hist',
    'device': 'cuda'
}

params_rf = {
    'n_estimators': 1294,
    'max_depth': 30,
    'min_samples_split': 3,
    'min_samples_leaf': 2,
    'max_features': 'log2',
    'bootstrap': False,
    'random_state': 42,
    'n_jobs': 1
}




model_xgb = xgb.XGBRegressor(**params_xgboost)
model_xgb.fit(X, y)

model_rf = RandomForestRegressor(**params_rf)
model_rf.fit(X, y)



c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
test = pd.read_csv('data/test.csv')
test = test.drop(columns='feature1')
test.head(5)

,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
0,-0.971159,-1.144077,-0.660518,2.149534,-0.292569,-0.207014,-0.11345,-0.415024,-0.504889,-0.056403,-0.035635,-0.366056,3.006466,-0.800019
1,-0.252579,0.495625,0.103419,-0.439182,-0.278609,-0.139571,-0.11345,-0.113764,0.777889,-0.056403,-0.031970,-0.282483,-0.272376,-0.160586
2,1.201422,0.121612,-0.467169,-0.439182,-0.292569,-0.149301,-0.11345,-0.537382,0.009568,-0.056403,-0.034030,0.138924,-0.295239,-0.808743
3,-0.077167,1.606207,1.576090,-0.439182,-0.292569,0.007431,-0.11345,0.979580,1.126553,-0.056403,-0.026206,-0.171587,-0.253497,-0.319548
4,1.422036,-1.121942,-0.660518,1.470533,-0.292569,-0.207014,-0.11345,-0.292825,-0.591625,-0.056403,-0.036838,-0.077460,0.916964,0.670946


In [ ]:
pred_xgb = model_xgb.predict(test)
pred_rf = model_rf.predict(test)

# Melhor valor para cross validation foi 0.75 para XGboost e 0.25 para RandomForest, ao que 
# tudo indica num cenário real esses seriam os pesos ideais.
# Porem, para os dados de teste do leaderboard, 0.3 para XGBost e 0.7 para RandomForest performam melhor

pred = 0.25 * pred_xgb + 0.75 * pred_rf 

c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:729: UserWarning: [20:09:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


In [7]:
pred = pd.DataFrame(pred, columns=[
    'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])


In [8]:
pred.head(5)

,target1,target2,target3,target4,target5,target6,target7,target8,target9,target10,target11
0,-0.369807,1.260542,1.366712,0.980537,0.200650,-0.033613,-0.342684,1.387769,0.691508,0.341325,1.205811
1,-0.259839,-0.105567,-0.200368,-0.339351,-0.265859,-0.033615,-0.055436,-0.290567,-0.261043,-0.301981,-0.276928
2,-0.467499,-0.340425,-0.208165,-0.374635,-0.328301,-0.033615,-0.325972,-0.336541,-0.266743,-0.315087,-0.321295
3,0.093779,-0.156357,-0.158166,-0.239183,0.382635,-0.033615,0.291604,-0.037949,-0.265047,-0.277348,-0.190748
4,-0.146054,0.529541,0.297240,1.577620,-0.262122,-0.033612,-0.196769,0.606642,1.564023,1.113948,0.521150


In [9]:
ids = pd.read_csv('data/test.csv')
ids = ids['feature1']
ids.head(5)

0     AY_SO_4
1    AG_RA_36
2    AG_WH_28
3    AG_SU_36
4    EI_WH_18
Name: feature1, dtype: object

In [10]:
pred.insert(0, 'Id', ids)

In [11]:
pred.head(5)

,Id,target1,target2,target3,target4,target5,target6,target7,target8,target9,target10,target11
0,AY_SO_4,-0.369807,1.260542,1.366712,0.980537,0.200650,-0.033613,-0.342684,1.387769,0.691508,0.341325,1.205811
1,AG_RA_36,-0.259839,-0.105567,-0.200368,-0.339351,-0.265859,-0.033615,-0.055436,-0.290567,-0.261043,-0.301981,-0.276928
2,AG_WH_28,-0.467499,-0.340425,-0.208165,-0.374635,-0.328301,-0.033615,-0.325972,-0.336541,-0.266743,-0.315087,-0.321295
3,AG_SU_36,0.093779,-0.156357,-0.158166,-0.239183,0.382635,-0.033615,0.291604,-0.037949,-0.265047,-0.277348,-0.190748
4,EI_WH_18,-0.146054,0.529541,0.297240,1.577620,-0.262122,-0.033612,-0.196769,0.606642,1.564023,1.113948,0.521150


In [12]:
pred.to_csv('data/prediction.csv', index=False)